In [6]:
import torch
import numpy as np
from torch.utils.data import DataLoader   
from SpeechDataGenerator import SpeechDataGenerator
import torch.nn as nn
import os
import numpy as np
from torch import optim
import argparse
from models.x_vector_Indian_LID import X_vector
from sklearn.metrics import accuracy_score
from utils.utils import speech_collate
import torch.nn.functional as F

In [7]:
### Data related
dataset_train = SpeechDataGenerator(manifest='meta/training_feat.txt',mode='train')
dataloader_train = DataLoader(dataset_train, num_workers=40, batch_size=256,shuffle=False,collate_fn=speech_collate) 

In [2]:
class TinyModel(torch.nn.Module):
    
    def __init__(self,bits = 128, checksum = 4):
        super(TinyModel, self).__init__()

        self.bits = bits
        self.checksum = checksum

        self.layer1 = nn.ModuleList([torch.nn.Sigmoid() for i in range(bits // checksum)])
        self.linears2 = nn.ModuleList([torch.nn.Linear(checksum, 1) for i in range(bits // checksum)])
        self.layer3 = torch.nn.Sigmoid()
    def forward(self, x):
        out = []
        for i in range(self.bits//self.checksum):
            xs = self.layer1[i](x[i*self.checksum:(i+1)*self.checksum])
            out.append(self.linears2[i](xs))
        out = torch.Tensor(out)
        out = self.layer3(out)
        return out

In [3]:
model = TinyModel()

In [4]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0, betas=(0.9, 0.98), eps=1e-9)
loss_fun = nn.CrossEntropyLoss()

In [5]:

def train(dataloader_train,epoch):
    train_loss_list=[]
    full_preds=[]
    full_gts=[]
    model.train()
    for i_batch, sample_batched in enumerate(dataloader_train):
    
        features = torch.from_numpy(np.asarray([torch_tensor.numpy().T for torch_tensor in sample_batched[0]])).float()
        labels = torch.from_numpy(np.asarray([torch_tensor[0].numpy() for torch_tensor in sample_batched[1]]))
        features.requires_grad = True
        optimizer.zero_grad()
        pred_logits,x_vec = model(features)
        #### CE loss
        loss = loss_fun(pred_logits,labels)
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())
        
        predictions = np.argmax(pred_logits.detach().cpu().numpy(),axis=1)
        for pred in predictions:
            full_preds.append(pred)
        for lab in labels.detach().cpu().numpy():
            full_gts.append(lab)
            
    mean_acc = accuracy_score(full_gts,full_preds)
    mean_loss = np.mean(np.asarray(train_loss_list))
    print('Total training loss {} and training Accuracy {} after {} epochs'.format(mean_loss,mean_acc,epoch))